In [1]:
import sys
sys.path.append('../rai/rai/ry')
import numpy as np
import libry as ry

In [2]:
# load and display a configuration
C = ry.Config()
C.addFile('../test/boxProblem.g')
D = C.view()
C.getFrameNames()

['base',
 'box',
 'boxBo',
 'boxLe',
 'boxBa',
 'ballR',
 'block',
 'target',
 'boxBa>target']

In [3]:
# create a standard path optimization problem with 4 phases, each of .2 seconds, and 10 steps
komo = C.komo_path(5., 10, .2, False)

In [4]:
# create tau-DOFs (time intervals) in the optimization problem and add respective constraints
komo.addTimeOptimization()

In [5]:
# take one object, and first 'switch' it to become freely movable without costs,
# then, starting at phase .7, make it dynamic (imposing NE equations)
obj = 'ballR'
#komo.addSwitch_magic(.0, 'base', obj)
komo.makeObjectsFree(["ballR"]);

In [6]:
# (old) a way to constrain the object position to be 'before' the box before phase 0.2
#komo.addObjective(time=[0., .2], type=OT.ineq, feature=FS.position, frames=[obj], scaleTrans=[[0.,1.,0.]], target=[1.])

In [7]:
# objectives for collision (permanent) and target (at phase-time 4.)
komo.addObjective(type=ry.OT.sos, feature=ry.FS.accumulatedCollisions)
komo.addObjective(time=[5.], type=ry.OT.eq, feature=ry.FS.distance, frames=['target', obj], scale=[3e1])

In [8]:
# simple structure of this solution: bounce on the bottom of the box at times 1, 2, and 3
komo.addSwitch_dynamicTrans(1., -1., 'base', obj)
komo.addInteraction_elasticBounce(2., "boxBo", obj, .8)
komo.addInteraction_elasticBounce(3., "boxBo", obj, .8)
komo.addInteraction_elasticBounce(4., "boxBo", obj, .8)

In [9]:
komo.optimize(True)

RuntimeError: frame.cpp:ensure_X:123(-2) CHECK failed: '_state_X_isGood' 

In [12]:
komoView = komo.view()

In [13]:
komo.display()

In [14]:
# read out solution: the full frame path, the tau path (time optimization), list of interaction forces
obj_path = komo.getPathFrames([obj])
tau_path = komo.getPathTau()
forces = komo.getForceInteractions()
forces

[{'at': 19,
  'force': [-2.2352977408443163e-09, 0.11789843163332533, -0.3860869361065004],
  'from': 'boxBo',
  'poa': [0.3314275866134213, -1.2715301345286307, 0.6483042753292537],
  'to': 'ballR'},
 {'at': 29,
  'force': [-8.190696335792792e-08,
   9.121055796959313e-07,
   -0.3436376897835519],
  'from': 'boxBo',
  'poa': [0.2448618044745193, -0.2777773442664518, 0.6499978386269324],
  'to': 'ballR'},
 {'at': 39,
  'force': [-2.140383693503435e-08,
   7.371444240614352e-07,
   -0.28949680972963204],
  'from': 'boxBo',
  'poa': [0.17050896376521768, 0.5762642788557996, 0.6500023007228017],
  'to': 'ballR'}]

Conversion from steps to phase and time:
```
phase=0 is the FIXED initial configuration and corresponds to step=-1
step=0 is the first configuration s.t. optimization
\tau(s) gives the time between step s-1 and step s
phase(step s) = (s+1)/stepsPerPhase
time(step s) = sum i=0:s tau(i)
```

In [15]:
D=0
komoView=0
komo=0
C=0